In [5]:
import torch
from torch.autograd import Variable

"""
A fully-connected ReLU network with one hidden layer, trained to predict y from x
by minimizing squared Euclidean distance.
This implementation uses the nn package from PyTorch to build the network.
PyTorch autograd makes it easy to define computational graphs and take gradients,
but raw autograd can be a bit too low-level for defining complex neural networks;
this is where the nn package can help. The nn package defines a set of Modules,
which you can think of as a neural network layer that has produces output from
input and may have some trainable weights.
"""

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Variables for its weight and bias.
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(size_average=False)

learning_rate = 1e-4
for t in range(2):
  # Forward pass: compute predicted y by passing x to the model. Module objects
  # override the __call__ operator so you can call them like functions. When
  # doing so you pass a Variable of input data to the Module and it produces
  # a Variable of output data.
  y_pred = model(x)

  # Compute and print loss. We pass Variables containing the predicted and true
  # values of y, and the loss function returns a Variable containing the loss.
  loss = loss_fn(y_pred, y)
  print(t, loss.data[0])
  
  # Zero the gradients before running the backward pass.
  model.zero_grad()

  # Backward pass: compute gradient of the loss with respect to all the learnable
  # parameters of the model. Internally, the parameters of each Module are stored
  # in Variables with requires_grad=True, so this call will compute gradients for
  # all learnable parameters in the model.
  loss.backward()

  # Update the weights using gradient descent. Each parameter is a Variable, so
  # we can access its data and gradients like we did before.
  for param in model.parameters():
    print("param.grad.data:",param.grad.data)
    param.data -= learning_rate * param.grad.data



(0, 659.0184326171875)
('param.grad.data:', 
 7.4433e-01 -7.2614e-01 -1.3192e+00  ...   5.3341e+00 -1.0674e-01 -5.2734e-01
 1.5793e+00 -3.4203e+00  6.8150e-01  ...  -1.0096e+00  1.4132e-01 -3.1041e-01
 2.2492e+00  9.6634e-01 -1.6080e+00  ...  -1.5044e-01 -2.5599e-01 -1.1852e+00
                ...                   ⋱                   ...                
-1.1388e+00  1.7256e+00 -1.2905e+00  ...  -6.1727e-01 -1.3786e-01  1.5557e+00
-2.3826e-01  2.5020e+00  2.5499e+00  ...  -5.6862e+00 -2.3627e+00  7.6780e-01
 1.9927e-01  1.2144e+00 -1.2210e+00  ...  -4.4220e-01 -1.9949e-01  5.8797e-01
[torch.FloatTensor of size 100x1000]
)
('param.grad.data:', 
 1.5334
 3.6870
-2.4351
-0.6471
 1.2608
 0.3858
 2.4086
-0.2965
 0.5493
 3.1871
-0.1990
-2.8714
-2.6718
-0.0858
-1.9341
-1.0703
 1.0302
-3.5466
 3.2375
 3.0162
 2.8786
 2.0190
 1.3694
 7.0514
-0.9801
 2.6815
 4.0489
-0.9153
 6.8797
 2.9408
-0.7775
 0.2176
-0.1786
 1.8134
 1.4138
 2.1126
-3.4032
 1.4905
 2.3443
-0.2541
 2.7077
 3.3078
 0.4223
-0.8

In [16]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x = torch.randn((2, 2))
y = torch.randn((2, 2))
z = x + y  # These are Tensor types, and backprop would not be possible

var_x = autograd.Variable(x)
var_y = autograd.Variable(y)
# var_z contains enough information to compute gradients, as we saw above
var_z = var_x + var_y
print(var_z.creator)

var_z_data = var_z.data  # Get the wrapped Tensor object out of var_z...
# Re-wrap the tensor in a new variable
new_var_z = autograd.Variable(var_z_data)

# ... does new_var_z have information to backprop to x and y?
# NO!
print(new_var_z.creator)
# And how could it?  We yanked the tensor out of var_z (that is
# what var_z.data is).  This tensor doesn't know anything about
# how it was computed.  We pass it into new_var_z, and this is all the
# information new_var_z gets.  If var_z_data doesn't know how it was
# computed, theres no way new_var_z will.
# In essence, we have broken the variable away from its past history

None


In [28]:
#not sequential nn
#teh layers are in nn.Linear. 

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2, True)
        self.fc2 = nn.Linear(2, 1, True)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
inputs = list(map(lambda s: Variable(torch.Tensor([s])), [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
]))
targets = list(map(lambda s: Variable(torch.Tensor([s])), [
    [0],
    [1],
    [1],
    [0]
]))


criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

params = list(net.fc1.parameters())
print("params fc1:", params)
print("Training loop:")
for idx in range(0, 2):
    for input, target in zip(inputs, targets):
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input)
        print("output:",output)
        loss = criterion(output, target)
        print("loss:",loss)
        loss.backward()
        optimizer.step()    # Does the update
    if idx % 5000 == 0:
        print("Epoch {: >8} Loss: {}".format(idx, loss.data.numpy()[0]))



print("")
print("Final results:")
for input, target in zip(inputs, targets):
    output = net(input)
    print("Input:[{},{}] Target:[{}] Predicted:[{}] Error:[{}]".format(
        int(input.data.numpy()[0][0]),
        int(input.data.numpy()[0][1]),
        int(target.data.numpy()[0]),
        round(float(output.data.numpy()[0]), 4),
        round(float(abs(target.data.numpy()[0] - output.data.numpy()[0])), 4)
    ))

('params fc1:', [Parameter containing:
 0.2154  0.6046
-0.1962 -0.2153
[torch.FloatTensor of size 2x2]
, Parameter containing:
 0.1004
 0.3547
[torch.FloatTensor of size 2]
])
Training loop:
('output:', Variable containing:
-0.2383
[torch.FloatTensor of size 1x1]
)
('loss:', Variable containing:
1.00000e-02 *
  5.6769
[torch.FloatTensor of size 1]
)
('output:', Variable containing:
-0.2191
[torch.FloatTensor of size 1x1]
)
('loss:', Variable containing:
 1.4862
[torch.FloatTensor of size 1]
)
('output:', Variable containing:
-0.1945
[torch.FloatTensor of size 1x1]
)
('loss:', Variable containing:
 1.4267
[torch.FloatTensor of size 1]
)
('output:', Variable containing:
-0.1425
[torch.FloatTensor of size 1x1]
)
('loss:', Variable containing:
1.00000e-02 *
  2.0310
[torch.FloatTensor of size 1]
)
Epoch        0 Loss: 0.0203102119267
('output:', Variable containing:
-0.1459
[torch.FloatTensor of size 1x1]
)
('loss:', Variable containing:
1.00000e-02 *
  2.1290
[torch.FloatTensor of size 1]

In [36]:
in1 = torch.zeros(2,1)
in1[0]=0.1
in1[1]=0.5
print(in1)

weights = torch.zeros(2,2)
weights[0,0]=.15
weights[0,1]=.20
weights[1,0]=.25
weights[1,1]=.30
print(weights)




 0.1000
 0.5000
[torch.FloatTensor of size 2x1]


 0.1500  0.2000
 0.2500  0.3000
[torch.FloatTensor of size 2x2]



In [39]:
m = nn.Linear(2,1,bias=False)
#m.weight[0,0]=0.15
print(m.weight)
print(m.bias)
input = autograd.Variable(torch.randn(1, 2))
output=m(input)
print(output)

Parameter containing:
 0.5357 -0.3656
[torch.FloatTensor of size 1x2]

None
Variable containing:
 0.1326
[torch.FloatTensor of size 1x1]



In [41]:
v=Variable(torch.Tensor([0,0,0]),requires_grad=True)
h = v.register_hook(lambda grad:grad * 2)
v.backward(torch.Tensor([1,1,1]))
v.grad.data




 2
 2
 2
[torch.FloatTensor of size 3]

In [42]:
v=Variable(torch.Tensor([0,0,0]))
#h = v.register_hook(lambda grad:grad * 2)
v.backward(torch.Tensor([1,1,1]))
v.grad.data

RuntimeError: there are no graph nodes that require computing gradients